In [1]:
# Import necessary libraries

import pandas as pd
import requests


In [2]:
def eurostat_series(dataset, params, freq, value_name="value"):
    params = dict(params)
    params["format"] = "JSON"
    EUROSTAT_BASE = "https://ec.europa.eu/eurostat/api/dissemination/statistics/1.0/data"
    try:
        r = requests.get(f"{EUROSTAT_BASE}/{dataset}", params=params, timeout=30)
        if "sorry.ec.europa.eu" in r.url:
            raise ConnectionError(
                "Eurostat API unavailable (redirected to sorry.ec.europa.eu)."
            )
        r.raise_for_status()
        j = r.json()
    except (requests.RequestException, ValueError) as exc:
        raise ConnectionError(
            "Eurostat API unavailable or returned invalid JSON."
        ) from exc
    time_index = j["dimension"]["time"]["category"]["index"]
    values = j.get("value", {})
    df = (
        pd.DataFrame(
            [(t, values.get(str(i))) for t, i in time_index.items()],
            columns=["period", value_name],
        )
        .dropna()
        .sort_values("period")
    )
    periods = df.pop("period").astype(str)
    if freq == "M":
        periods = periods.str.replace(r"^(\d{4})M(\d{2})$", r"\1-\2", regex=True)
    df["time"] = (
        pd.PeriodIndex(periods, freq=freq).to_timestamp(how="end").normalize()
    )
    return df.reset_index(drop=True)


def eurostat_components(
    dataset,
    base_params,
    items,
    freq,
    dim="na_item",
    how="inner",
):
    if isinstance(items, (list, tuple)):
        items = {item: item for item in items}
    frames = []
    for item_code, col_name in items.items():
        params = dict(base_params, **{dim: item_code})
        frames.append(
            eurostat_series(dataset, params, freq, value_name=col_name)
        )
    if not frames:
        return pd.DataFrame(columns=["time"])
    df = frames[0]
    for frame in frames[1:]:
        df = df.merge(frame, on="time", how=how)
    return df.sort_values("time").reset_index(drop=True)


In [3]:
base_params = dict(geo="PT", unit="CP_MEUR")

# GDP and expenditure components (annual, current prices)
components = [
    "B1GQ",

    "P31_S14_S15",
    "P31_S13",
    "P32_S13",
    "P51G",
    "P52_P53",
    "P6",
    "P7",

    "D1",
    "B2A3G",
    "D2",
    "D3",

    "B1G",
    "D21",
    "D31",]

y = eurostat_components("nama_10_gdp", base_params, components, freq="Y")
y["gdp_eo_exp"] = y["B1GQ"] - (
    y["P31_S14_S15"]+
    y["P31_S13"]+
    y["P32_S13"]+
    y["P51G"]+y["P52_P53"]+
    y["P6"]-y["P7"])

y["gdp_eo_inc"] = y["B1GQ"] - (
    y["D1"]+
    y["B2A3G"]+
    y["D2"]-y["D3"])

y["gdp_eo_prod"] = y["B1GQ"] - (
    y["B1G"]+
    y["D21"]-y["D31"])

# B1G by industry (annual, current prices)
nace_components = [
    "TOTAL",
    "A",
    "B-E",
    "F",
    "G-I",
    "J",
    "K",
    "L",
    "M_N",
    "O-Q",
    "R-U",
]

b1g_by_nace = eurostat_components(
    "nama_10_a10",
    dict(base_params, na_item="B1G"),
    nace_components,
    freq="Y",
    dim="nace_r2",
    how="outer",
)


In [4]:
import plotly.graph_objects as go

df = y.dropna(subset=["B1GQ"]).sort_values("time").copy()
df["public_consumption"] = df["P31_S13"] + df["P32_S13"]
df["net_exports"] = df["P6"] - df["P7"]
df["time"] = df["time"] - pd.DateOffset(months=6)

components = [
    ("P31_S14_S15", "Consumo privado (P31_S14_S15)"),
    ("public_consumption", "Consumo público (P31_S13+P32_S13)"),
    ("P51G", "Investimento físico (P51G)"),
    ("P52_P53", "Variação de existências e valorações (P52_P53)"),
    ("net_exports", "Exportações líquidas (P6-P7)"),
]

colors = {
    "P31_S14_S15": "rgba(37, 99, 235, 0.6)",
    "public_consumption": "rgba(16, 185, 129, 0.6)",
    "P51G": "rgba(249, 115, 22, 0.6)",
    "P52_P53": "rgba(239, 68, 68, 0.6)",
    "net_exports": "rgba(107, 114, 128, 0.6)",
}

for col, _ in components:
    df[f"{col}_share"] = df[col] / df["B1GQ"] * 100

fig = go.Figure()
for col, label in components:
    fig.add_trace(
        go.Bar(
            x=df["time"],
            y=df[f"{col}_share"],
            name=label,
            marker_color=colors[col],
        )
    )

fig.update_layout(
    barmode="relative",
    template="plotly_white",
    title="Portugal: componentes da despesa do PIB (% do PIB)",
    title_x=0.5,
    legend_title_text="",
    xaxis_title="",
    yaxis_title="",
    legend=dict(orientation="h", x=0.5, y=-0.2, xanchor="center", yanchor="top", bgcolor="rgba(0,0,0,0)", borderwidth=0),
    margin=dict(t=90),
)
fig.update_xaxes(dtick="M36", tickformat="%Y", hoverformat="%Y")
fig.update_yaxes(tickformat=".1f", ticksuffix="%")
fig.show()


In [5]:
import plotly.graph_objects as go

df = y.dropna(subset=["B1GQ"]).sort_values("time").copy()
df["D2_D3"] = df["D2"] - df["D3"]
df["time"] = df["time"] - pd.DateOffset(months=6)

components = [
    ("D1", "Remuneração dos empregados (D1)"),
    ("B2A3G", "Excedente bruto de exploração e rendimento misto (B2A3G)"),
    ("D2_D3", "Impostos líquidos sobre a produção e importações (D2-D3)"),
]

colors = {
    "D1": "rgba(37, 99, 235, 0.6)",
    "B2A3G": "rgba(16, 185, 129, 0.6)",
    "D2_D3": "rgba(249, 115, 22, 0.6)",
}

for col, _ in components:
    df[f"{col}_share"] = df[col] / df["B1GQ"] * 100

fig = go.Figure()
for col, label in components:
    fig.add_trace(
        go.Bar(
            x=df["time"],
            y=df[f"{col}_share"],
            name=label,
            marker_color=colors[col],
        )
    )

fig.update_layout(
    barmode="relative",
    template="plotly_white",
    title="Portugal: componentes do PIB pela ótica do rendimento (% do PIB)",
    title_x=0.5,
    legend_title_text="",
    xaxis_title="",
    yaxis_title="",
    legend=dict(orientation="h", x=0.5, y=-0.2, xanchor="center", yanchor="top", bgcolor="rgba(0,0,0,0)", borderwidth=0),
)
fig.update_xaxes(dtick="M36", tickformat="%Y", hoverformat="%Y")
fig.update_yaxes(tickformat=".1f", ticksuffix="%")
fig.show()


In [6]:
import plotly.graph_objects as go

df = (
    b1g_by_nace.merge(
        y[["time", "B1GQ", "D21", "D31"]],
        on="time",
        how="inner",
    )
    .sort_values("time")
    .copy()
)

df["net_taxes"] = df["D21"] - df["D31"]
df["time"] = df["time"] - pd.DateOffset(months=6)

sector_components = [
    c for c in nace_components if c != "TOTAL" and c in df.columns
]

sector_labels = {
    "A": "Agric., silvic. e pesca (A)",
    "B-E": "Indústria (B-E)",
    "F": "Construção (F)",
    "G-I": "Comércio, transp. e aloj. (G-I)",
    "J": "Informação e comunicação (J)",
    "K": "Atividades fin. e de seguros (K)",
    "L": "Atividades imobiliárias (L)",
    "M_N": "Atividades prof. e admin. (M_N)",
    "O-Q": "Admin. púb., educ. e saúde (O-Q)",
    "R-U": "Artes e outros serviços (R-U)",
}

components = [(c, sector_labels.get(c, c)) for c in sector_components]
components.append(("net_taxes", "Impostos líq. nos prod. (D21-D31)"))

palette = [
    "rgba(37, 99, 235, 0.6)",
    "rgba(16, 185, 129, 0.6)",
    "rgba(249, 115, 22, 0.6)",
    "rgba(239, 68, 68, 0.6)",
    "rgba(107, 114, 128, 0.6)",
    "rgba(14, 165, 233, 0.6)",
    "rgba(168, 85, 247, 0.6)",
    "rgba(234, 179, 8, 0.6)",
    "rgba(20, 184, 166, 0.6)",
    "rgba(236, 72, 153, 0.6)",
    "rgba(148, 163, 184, 0.6)",
]

palette = palette * ((len(components) // len(palette)) + 1)
colors = {c: palette[i] for i, (c, _) in enumerate(components)}

for col, _ in components:
    df[f"{col}_share"] = df[col] / df["B1GQ"] * 100

fig = go.Figure()
for col, label in components:
    fig.add_trace(
        go.Bar(
            x=df["time"],
            y=df[f"{col}_share"],
            name=label,
            marker_color=colors[col],
        )
    )

fig.update_layout(
    barmode="relative",
    template="plotly_white",
    title="Portugal: componentes do PIB pela ótica da produção (% do PIB)",
    title_x=0.5,
    legend_title_text="",
    xaxis_title="",
    yaxis_title="",
    legend=dict(x=1.02, y=0.5, xanchor="left", yanchor="middle", bgcolor="rgba(0,0,0,0)", borderwidth=0),
)
fig.update_xaxes(dtick="M36", tickformat="%Y", hoverformat="%Y")
fig.update_yaxes(tickformat=".1f", ticksuffix="%")
fig.show()


In [7]:
# Summary table for selected years

years = [2004, 2014, 2024]


def series_by_year(df, col, years):
    tmp = df.copy()
    tmp["year"] = tmp["time"].dt.year
    tmp = tmp[tmp["year"].isin(years)]
    return tmp.set_index("year")[col].reindex(years)


def add_row(rows, approach, name, code, values, years):
    row = {
        "Abordagem": approach,
        "Componente": name,
        "Código": code,
    }
    for yr in years:
        row[str(yr)] = values.loc[yr]
    rows.append(row)


gdp_series = series_by_year(y[["time", "B1GQ"]], "B1GQ", years)

rows = []
#add_row(rows, "-", "PIB a preços correntes", "B1GQ", gdp_series, years)

exp_df = y[
    [
        "time",
        "B1GQ",
        "P31_S14_S15",
        "P31_S13",
        "P32_S13",
        "P51G",
        "P52_P53",
        "P6",
        "P7",
    ]
].copy()
exp_df["public_consumption"] = exp_df["P31_S13"] + exp_df["P32_S13"]
exp_df["net_exports"] = exp_df["P6"] - exp_df["P7"]

exp_components = [
    ("P31_S14_S15", "Consumo privado", "P31_S14_S15"),
    ("public_consumption", "Consumo público", "P32_S13+P31_S13"),
    ("P51G", "Investimento físico", "P51G"),
    ("P52_P53", "Variação de existências e valorações", "P52_P53"),
    ("net_exports", "Exportações líquidas", "P6-P7"),
]

for col, name, code in exp_components:
    values = series_by_year(exp_df, col, years) / gdp_series * 100
    add_row(rows, "Despesa", name, code, values, years)

inc_df = y[["time", "B1GQ", "D1", "B2A3G", "D2", "D3"]].copy()
inc_df["D2_D3"] = inc_df["D2"] - inc_df["D3"]

inc_components = [
    ("D1", "Remuneração dos empregados", "D1"),
    (
        "B2A3G",
        "Excedente bruto de exploração e rendimento misto",
        "B2A3G",
    ),
    (
        "D2_D3",
        "Impostos líquidos sobre a produção e importações",
        "D2-D3",
    ),
]

for col, name, code in inc_components:
    values = series_by_year(inc_df, col, years) / gdp_series * 100
    add_row(rows, "Rendimento", name, code, values, years)

prod_df = (
    b1g_by_nace.merge(
        y[["time", "B1GQ", "D21", "D31"]],
        on="time",
        how="inner",
    )
    .sort_values("time")
    .copy()
)
prod_df["net_taxes"] = prod_df["D21"] - prod_df["D31"]

sector_components = [
    c for c in nace_components if c != "TOTAL" and c in prod_df.columns
]

sector_labels = {
    "A": "Agricultura, silvicultura e pesca",
    "B-E": "Indústria",
    "F": "Construção",
    "G-I": "Comércio, transportes e alojamento",
    "J": "Informação e comunicação",
    "K": "Atividades financeiras e de seguros",
    "L": "Atividades imobiliárias",
    "M_N": "Atividades profissionais e administrativas",
    "O-Q": "Administração pública, educação e saúde",
    "R-U": "Artes e outros serviços",
}

for code in sector_components:
    name = sector_labels.get(code, code)
    values = series_by_year(prod_df, code, years) / gdp_series * 100
    add_row(rows, "Produção", name, code, values, years)

values = series_by_year(prod_df, "net_taxes", years) / gdp_series * 100
add_row(
    rows,
    "Produção",
    "Impostos líquidos sobre os produtos",
    "_D21-D31",
    values,
    years,
)

table = pd.DataFrame(rows)
table = table.round(1)

table = table.sort_values(["Abordagem", "Código"])
table = table.set_index(["Abordagem", "Componente", "Código"])

# Zebra pattern with distinct palettes per Abordagem
zebra_palettes = {
    "Despesa": ("#e8f2ff", "#f5f9ff"),
    "Produção": ("#e9f7ef", "#f4fbf7"),
    "Rendimento": ("#fff1e5", "#fff8f1"),
}
default_palette = ("#f2f2f2", "#fafafa")

def zebra_by_abordagem(row):
    abordagem = row.name[0] if isinstance(row.name, tuple) else row.name
    palette = zebra_palettes.get(abordagem, default_palette)
    row_pos = zebra_by_abordagem._counters.get(abordagem, 0)
    zebra_by_abordagem._counters[abordagem] = row_pos + 1
    color = palette[row_pos % 2]
    return [f"background-color: {color}"] * len(row)

zebra_by_abordagem._counters = {}

row_colors = []
row_counters = {}
for idx in table.index:
    abordagem = idx[0] if isinstance(idx, tuple) else idx
    palette = zebra_palettes.get(abordagem, default_palette)
    pos = row_counters.get(abordagem, 0)
    row_counters[abordagem] = pos + 1
    row_colors.append(palette[pos % 2])

def zebra_rows(data):
    return pd.DataFrame(
        [[f"background-color: {c}"] * data.shape[1] for c in row_colors],
        index=data.index,
        columns=data.columns,
    )

def zebra_index(_):
    return [f"background-color: {c}" for c in row_colors]


num_cols = table.select_dtypes(include="number").columns

styled = (
    table.style
    .apply(zebra_rows, axis=None)
    .apply_index(zebra_index, axis=0)
    .format("{:.1f}", subset=num_cols)
    .set_properties(subset=pd.IndexSlice[:, :], **{"text-align": "right"})
    .set_table_styles([
        {"selector": "th", "props": [("text-align", "left")]},
    ])
    .set_table_styles([
        {"selector": "th.row_heading", "props": [("text-align", "left")]},
        {"selector": "th.index_name", "props": [("text-align", "left")]},
    ], overwrite=False)
)

styled


In [8]:
# Labor income share (annual)
labor_params = dict(geo="PT", unit="THS_PER")

labor_inputs = eurostat_components(
    "nama_10_pe",
    labor_params,
    ["EMP_DC", "SAL_DC"],
    freq="Y",
)

labor_share = (
    y.merge(labor_inputs, on="time", how="inner")
    .sort_values("time")
    .copy()
)

labor_share["GVAfp"] = labor_share["B1GQ"] - (
    labor_share["D2"] - labor_share["D3"]
)
labor_share["LIS_na"] = labor_share["D1"] / labor_share["GVAfp"]
labor_share["LIS_a"] = labor_share["LIS_na"] * (
    labor_share["EMP_DC"] / labor_share["SAL_DC"]
)
labor_share["LIS_bad"] = labor_share["D1"] / labor_share["B1GQ"]

labor_share = labor_share[
    [
        "time",
        "B1GQ",
        "D1",
        "D2",
        "D3",
        "GVAfp",
        "EMP_DC",
        "SAL_DC",
        "LIS_na",
        "LIS_a",
        "LIS_bad",
    ]
]


In [9]:
import plotly.express as px

df = (
    labor_share[["time", "LIS_na", "LIS_a", "LIS_bad"]]
    .dropna()
    .sort_values("time")
    .copy()
)
df["time"] = df["time"] - pd.DateOffset(months=6)

df_long = (
    df.rename(
        columns={
            "LIS_na": "não ajustada: D1/GVAfp",
            "LIS_a": "ajustada: (D1/GVAfp)(EMP/SAL)",
            "LIS_bad": "ingénua/incorrecta: D1/PIB",
        }
    )
    .melt(
        id_vars="time",
        value_vars=[
            "ajustada: (D1/GVAfp)(EMP/SAL)",
            "não ajustada: D1/GVAfp",
            "ingénua/incorrecta: D1/PIB",
        ],
        var_name="series",
        value_name="lis",
    )
)

df_long["lis"] = df_long["lis"] * 100

fig = px.line(
    df_long,
    x="time",
    y="lis",
    color="series",
    title="Portugal: parte do rendimento do trabalho (labor share)",
    template="plotly_white",
    color_discrete_map={
        "não ajustada: D1/GVAfp": "red",
        "ajustada: (D1/GVAfp)(EMP/SAL)": "blue",
        "ingénua/incorrecta: D1/PIB": "orange",
    },
    markers=True,
    line_shape="linear",
)

fig.update_traces(
    line_width=2.5,
    marker_size=6,
    marker_color="white",
    marker_symbol="circle",
    marker_line_width=2.0,
)

fig.update_layout(
    title_x=0.5,
    legend_title_text="",
    xaxis_title="",
    yaxis_title="",
    legend=dict(
        x=0.98,
        y=0.02,
        xanchor="right",
        yanchor="bottom",
        bgcolor="rgba(0,0,0,0)",
        borderwidth=0,
    ),
)
fig.update_xaxes(dtick="M36", tickformat="%Y", hoverformat="%Y")
fig.update_yaxes(tickformat=".1f", ticksuffix="%", range=[25, 85])
fig.show()


In [10]:
# OECD IOTs Total Table (latest Portugal)
import io
import re
import zipfile

zip_url = "https://stats.oecd.org/wbos/fileview2.aspx?IDFile=67d903bc-edff-463c-aeeb-52d711731221"

zip_resp = requests.get(zip_url, timeout=60)
zip_resp.raise_for_status()

z = zipfile.ZipFile(io.BytesIO(zip_resp.content))
prt_files = [name for name in z.namelist() if name.startswith("PRT") and name.endswith("ttl.csv")]
if not prt_files:
    raise ValueError("No PRT ttl files found in IOTs_TTL.zip")

years = []
for name in prt_files:
    match = re.search(r"(\d{4})", name)
    if match:
        years.append(int(match.group(1)))
latest_year = max(years)
latest_file = [name for name in prt_files if str(latest_year) in name][0]

with z.open(latest_file) as f:
    iot_prt_ttl = pd.read_csv(f, index_col=0)

iot_prt_ttl.index.name = "ROW"



In [11]:
# Aggregate the OECD Total IOT (TTL) to NACE letter level (A..T)
final_demand_cols = [
    "HFCE",
    "NPISH",
    "GGFC",
    "GFCF",
    "INVNT",
    "DPABR",
    "CONS_NONRES",
    "EXPO",
    "IMPO",
    "TOTAL",
]

industry_cols = [c for c in iot_prt_ttl.columns if c not in final_demand_cols]
industry_col_groups = [c[0] for c in industry_cols]

# First aggregate industry columns to letters; keep final demand columns as-is.
iot_cols_letter = pd.concat(
    [
        iot_prt_ttl[industry_cols].T.groupby(industry_col_groups, sort=False).sum().T,
        iot_prt_ttl[final_demand_cols],
    ],
    axis=1,
)

iot_cols_letter.index.name = "ROW"

# Then aggregate industry rows (TTL_*) to letters.
industry_rows = [
    r for r in iot_cols_letter.index if r.startswith("TTL_") and r != "TTL_INT_FNL"
]

ttl = iot_cols_letter.loc[industry_rows].copy()
ttl.index = [r.replace("TTL_", "")[0] for r in ttl.index]
ttl = ttl.groupby(level=0).sum()

other = iot_cols_letter.drop(index=industry_rows)

iot_prt_ttl_letter = pd.concat([ttl, other], axis=0)



In [12]:
# Aggregate letter-level industries into broad sectors

sector_order = ["A", "B-E", "F", "G-I", "J", "K", "L", "M-N", "O-Q", "R-U"]

letter_to_sector = {
    "A": "A",
    "B": "B-E",
    "C": "B-E",
    "D": "B-E",
    "E": "B-E",
    "F": "F",
    "G": "G-I",
    "H": "G-I",
    "I": "G-I",
    "J": "J",
    "K": "K",
    "L": "L",
    "M": "M-N",
    "N": "M-N",
    "O": "O-Q",
    "P": "O-Q",
    "Q": "O-Q",
    "R": "R-U",
    "S": "R-U",
    "T": "R-U",
    "U": "R-U",
}

industry_letters = [c for c in iot_prt_ttl_letter.columns if re.fullmatch(r"[A-Z]", str(c))]
letter_groups = [letter_to_sector.get(c, c) for c in industry_letters]

cols_grouped = iot_prt_ttl_letter[industry_letters].T.groupby(letter_groups, sort=False).sum().T
other_cols = iot_prt_ttl_letter.drop(columns=industry_letters)

iot_cols_grouped = pd.concat([cols_grouped, other_cols], axis=1)

industry_rows = [r for r in iot_cols_grouped.index if re.fullmatch(r"[A-Z]", str(r))]
rows = iot_cols_grouped.loc[industry_rows].copy()
rows.index = [letter_to_sector.get(r, r) for r in rows.index]
rows = rows.groupby(level=0, sort=False).sum()
other_rows = iot_cols_grouped.drop(index=industry_rows)

iot_prt_ttl_broad = pd.concat([rows, other_rows], axis=0)

# Aggregate final demand columns
final_demand_groups = {
    "cons*": ["HFCE", "NPISH", "DPABR"],
    "inv*": ["GFCF", "INVNT"],
    "exp*": ["CONS_NONRES", "EXPO"],
}

for new_col, cols in final_demand_groups.items():
    cols_present = [c for c in cols if c in iot_prt_ttl_broad.columns]
    if cols_present:
        iot_prt_ttl_broad[new_col] = iot_prt_ttl_broad[cols_present].sum(axis=1)

drop_cols = [c for cols in final_demand_groups.values() for c in cols]
iot_prt_ttl_broad = iot_prt_ttl_broad.drop(columns=[c for c in drop_cols if c in iot_prt_ttl_broad.columns])

final_order = [
    "cons*",
    "GGFC",
    "inv*",
    "exp*",
    "IMPO",
]
col_order = [s for s in sector_order if s in iot_prt_ttl_broad.columns]
final_order = [c for c in final_order if c in iot_prt_ttl_broad.columns]
other_col_order = [c for c in iot_prt_ttl_broad.columns if c not in col_order + final_order]
iot_prt_ttl_broad = iot_prt_ttl_broad[col_order + final_order + other_col_order]

row_order = [s for s in sector_order if s in iot_prt_ttl_broad.index]
other_row_order = [r for r in iot_prt_ttl_broad.index if r not in row_order]
iot_prt_ttl_broad = iot_prt_ttl_broad.loc[row_order + other_row_order]

# Indent selected index labels for display
indent4 = {"A", "B-E", "F", "G-I", "J", "K", "L", "M-N", "O-Q", "R-U",
           "IMP_OTHER", "TXS_INT_FNL", "LABR", "OTXS", "CFC", "NOPS"}
indent2 = {"TTL_INT_FNL", "VALU"}

def _indent_label(lbl):
    s = str(lbl)
    if s in indent4:
        return "    " + s
    if s in indent2:
        return "  " + s
    return s

iot_prt_ttl_broad.index = [_indent_label(idx) for idx in iot_prt_ttl_broad.index]



In [13]:
# style io table
import numpy as np

# Replace 0.0 with NaN for display
io_table = iot_prt_ttl_broad.copy()
io_table = io_table.replace(0.0, np.nan)

num_cols = io_table.select_dtypes(include="number").columns

styled_io = (
    io_table.style
    .format("{:.1f}", subset=num_cols, na_rep="")
    .set_properties(**{"font-size": "0.8em"})
    .set_properties(subset=pd.IndexSlice[:, :], **{"text-align": "right"})
    .set_table_styles([
        {"selector": "thead th", "props": [("font-size", "0.8em")]},
        {"selector": "th", "props": [("text-align", "right")]},
        {"selector": "th.row_heading", "props": [("text-align", "left"), ("font-size", "0.8em")]},
        {"selector": "th.index_name", "props": [("text-align", "left"), ("font-size", "0.8em")]},
    ])
    .set_table_styles([
        {"selector": "th.row_heading", "props": [("white-space", "pre"), ("font-size", "0.8em")]},
    ], overwrite=False)
)

styled_io



,A,B-E,F,G-I,J,K,L,M-N,O-Q,R-U,cons*,GGFC,inv*,exp*,IMPO,TOTAL
A,2234.7,6799.5,84.2,934.0,13.5,6.8,11.5,54.6,98.0,25.3,5333.6,8.3,576.4,1902.9,-4413.6,13669.6
B-E,3466.0,81375.6,9333.7,17540.6,1422.4,506.8,743.7,2502.5,6014.8,1237.9,44539.6,1137.7,12316.1,62638.0,-81129.8,163645.6
F,192.8,1623.9,5637.8,1186.8,156.2,95.6,1314.8,431.8,945.7,106.8,264.5,236.4,19947.8,83.3,-140.0,32084.3
G-I,1731.0,20877.0,2896.3,14539.4,1050.6,638.6,278.5,2252.2,3164.6,692.4,37721.6,2746.0,4695.5,34935.3,-22195.1,106024.1
J,52.5,1705.3,258.1,2353.0,4442.3,808.7,175.0,1878.6,1143.4,371.1,3774.9,288.5,4856.9,5039.6,-6050.4,21097.6
K,288.4,2961.7,401.3,3026.8,411.5,3055.5,1156.9,986.0,673.6,270.2,6138.5,281.6,288.0,1492.8,-2101.0,19331.8
L,21.3,684.2,187.1,2728.4,344.8,256.1,326.8,563.6,531.2,208.5,21279.7,60.8,4013.9,1107.9,-357.5,31956.9
M-N,373.0,7090.5,1852.2,7968.0,2061.8,1528.8,881.2,6214.0,2853.6,892.5,2439.0,317.6,2185.3,6811.8,-6326.5,37143.0
O-Q,38.2,595.1,180.2,652.7,166.9,118.9,79.5,371.2,1902.9,81.2,15524.7,38782.1,852.4,1234.6,-667.0,59913.5
R-U,17.3,242.4,51.9,354.8,151.3,56.0,21.2,186.5,185.7,832.0,8022.3,368.6,155.3,1018.4,-528.2,11135.4


In [ ]:
# Descritores (PT-PT) para os códigos do iot_prt_ttl_broad
import pandas as pd

# Descritores para códigos base (sem espaços de indentação)
iot_code_desc_pt = {
    # Setores (agregados)
    "A": "Agricultura, silvicultura e pesca",
    "B-E": "Indústrias extrativas e transformadoras; eletricidade, gás, água, saneamento e resíduos",
    "F": "Construção",
    "G-I": "Comércio, transportes, alojamento e restauração",
    "J": "Informação e comunicação",
    "K": "Atividades financeiras e de seguros",
    "L": "Atividades imobiliárias",
    "M-N": "Atividades profissionais, científicas e técnicas; atividades administrativas e de apoio",
    "O-Q": "Administração pública, educação, saúde e ação social",
    "R-U": "Artes, entretenimento e outros serviços; famílias e organizações internacionais",

    # Procura final (agregada)
    "cons*": "Consumo final das famílias e ISFLSF + compras diretas no estrangeiro (HFCE+NPISH+DPABR)",
    "GGFC": "Despesa de consumo final das administrações públicas",
    "inv*": "Investimento: FBCF + variação de existências (GFCF+INVNT)",
    "exp*": "Procura externa: consumo de não residentes + exportações (CONS_NONRES+EXPO)",
    "IMPO": "Importações",
    "TOTAL": "Total",

    # Outras rubricas / agregados da tabela
    "IMP_OTHER": "Importações (outras)",
    "TXS_INT_FNL": "Impostos líquidos de subsídios sobre produtos (consumo intermédio + procura final)",
    "TTL_INT_FNL": "Total (consumo intermédio + procura final)",
    "VALU": "Valor acrescentado bruto (VAB)",
    "LABR": "Remunerações dos trabalhadores",
    "OTXS": "Outros impostos líquidos de subsídios à produção",
    "CFC": "Consumo de capital fixo",
    "NOPS": "Excedente de exploração líquido",
    "OUTPUT": "Produção",
}

index_codes = pd.Index(iot_prt_ttl_broad.index.astype(str))
column_codes = pd.Index(iot_prt_ttl_broad.columns.astype(str))

base_index = index_codes.str.strip()
base_columns = column_codes.str.strip()
base_all = pd.Index(base_index.union(base_columns))

# Ordem preferencial por códigos base (sem indentação)
preferred_order = [
    "A",
    "B-E",
    "F",
    "G-I",
    "J",
    "K",
    "L",
    "M-N",
    "O-Q",
    "R-U",
    "cons*",
    "GGFC",
    "inv*",
    "exp*",
    "IMPO",
    "TOTAL",
    "IMP_OTHER",
    "TXS_INT_FNL",
    "TTL_INT_FNL",
    "VALU",
    "LABR",
    "OTXS",
    "CFC",
    "NOPS",
    "OUTPUT",
]

# Constrói lista única por código base, preservando a indentação (se existir na linha)
codes = []
for base in preferred_order:
    if base not in base_all:
        continue
    # Preferir o rótulo do índice (preserva indentação)
    idx_match = index_codes[base_index == base]
    display = idx_match[0] if len(idx_match) else base
    codes.append(display)

# Adiciona restantes códigos não previstos, mantendo indentação do índice se existir
remaining = [b for b in base_all if b not in preferred_order]
for base in remaining:
    idx_match = index_codes[base_index == base]
    display = idx_match[0] if len(idx_match) else base
    if display not in codes:
        codes.append(display)

code_base = [c.strip() for c in codes]

# Flags agregados por código base (linhas/colunas)
base_in_rows = set(base_index)
base_in_cols = set(base_columns)

iot_prt_ttl_broad_codebook_pt = pd.DataFrame(
    {
        "codigo": codes,
        "descricao_pt": [iot_code_desc_pt.get(b, b) for b in code_base],
        "em_linhas": [b in base_in_rows for b in code_base],
        "em_colunas": [b in base_in_cols for b in code_base],
    }
)


# Estilo do glossário
codebook_display = iot_prt_ttl_broad_codebook_pt.copy()
codebook_display = codebook_display.rename(
    columns={
        "codigo": "Código",
        "descricao_pt": "Descrição",
        "em_linhas": "Linhas",
        "em_colunas": "Colunas",
    }
)
codebook_display["Linhas"] = codebook_display["Linhas"].map({True: "sim", False: "não"})
codebook_display["Colunas"] = codebook_display["Colunas"].map({True: "sim", False: "não"})
codebook_display["Código"] = codebook_display["Código"].str.lstrip()


# Index principal: padrão (Linhas, Colunas)
codebook_display["Padrão"] = codebook_display["Linhas"] + "/" + codebook_display["Colunas"]
pattern_order = ["sim/sim", "não/sim", "sim/não"]
codebook_display["Padrão"] = pd.Categorical(codebook_display["Padrão"], categories=pattern_order, ordered=True)
codebook_display["_ord"] = codebook_display.groupby("Padrão", observed=False).cumcount()
codebook_display = codebook_display.set_index(["Padrão", "Código", "_ord"])
codebook_display = codebook_display.rename_axis(index={"Padrão": "Lin/Col"})
codebook_display = codebook_display.drop(columns=["Linhas", "Colunas"])

# Remover espaços iniciais do Código apenas para o índice

# Cores por grupo do índice Lin/Col
pattern_colors = {
    "sim/sim": ("#e8f2ff", "#f5f9ff"),
    "não/sim": ("#e9f7ef", "#f4fbf7"),
    "sim/não": ("#fff1e5", "#fff8f1"),
}
row_colors = []
row_counters = {}
for idx in codebook_display.index:
    group = idx[0] if isinstance(idx, tuple) else idx
    palette = pattern_colors.get(group, ("#f2f2f2", "#fafafa"))
    pos = row_counters.get(group, 0)
    row_counters[group] = pos + 1
    row_colors.append(palette[pos % 2])

def zebra_rows(data):
    return pd.DataFrame(
        [[f"background-color: {c}"] * data.shape[1] for c in row_colors],
        index=data.index,
        columns=data.columns,
    )

def zebra_index(_):
    return [f"background-color: {c}" for c in row_colors]

codebook_style = (
    codebook_display.style.hide(axis="index", level=2)
    .apply(zebra_rows, axis=None)
    .apply_index(zebra_index, axis=0)
    .set_table_styles([
        {"selector": "th", "props": [("text-align", "left")]},
        {"selector": "th.col_heading", "props": [("text-align", "left")]},
        {"selector": "th.row_heading", "props": [("text-align", "left")]},
        {"selector": "th.index_name", "props": [("text-align", "left")]},
    ], overwrite=False)
    .set_properties(subset=codebook_display.columns, **{"text-align": "right"})
    .set_properties(subset=["Descrição"], **{"text-align": "left"})
)

codebook_style




